In [ ]:
 ! pip install chembl_webresource_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
jsonschema 4.19.1 requires attrs>=22.2.0, but you have attrs 21.4.0 which is incompatible.
referencing 0.30.2 requires attrs>=22.2.0, but you have attrs 21.4.0 which is incompatible.


In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [ ]:
### *Search for Antibiotics*

In [ ]:
# Create a ChEMBL client
chembl_client = new_client

# Get Data related to Staph.
target = new_client.target.search('Staphylococcus aureus')
query = pd.DataFrame.from_dict(target)


In [ ]:
groups = list(query['target_chembl_id'])

In [ ]:
selected = groups[1]

In [ ]:
cols = new_client.activity.filter(target_chembl_id = selected).filter(standard_type  = "MIC").only(
    'molecule_chembl_id',
    'ligand_efficiency',
    'target_organism',
    'canonical_smiles',
    'standard_value',
    'standard_units',
    'standard_type',
    'parent_molecule_chembl_id',
    'target_pref_name'
)


In [ ]:
df = pd.DataFrame.from_dict(cols)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   canonical_smiles           905 non-null    object
 1   ligand_efficiency          0 non-null      object
 2   molecule_chembl_id         937 non-null    object
 3   parent_molecule_chembl_id  937 non-null    object
 4   standard_type              937 non-null    object
 5   standard_units             937 non-null    object
 6   standard_value             937 non-null    object
 7   target_organism            937 non-null    object
 8   target_pref_name           937 non-null    object
 9   type                       937 non-null    object
 10  units                      937 non-null    object
 11  value                      937 non-null    object
dtypes: object(12)
memory usage: 88.0+ KB


In [ ]:
df.drop(
    ['ligand_efficiency', 'parent_molecule_chembl_id','target_pref_name', 'type', 'units'],
    inplace = True,
    axis = 1)

In [ ]:
df.head(3)

,canonical_smiles,molecule_chembl_id,standard_type,standard_units,standard_value,target_organism,value
0,C=C[C@](C)(CC/C=C(\C)CC/C=C(\C)CC/C=C(/C)CO[C@...,CHEMBL2271377,MIC,ug.mL-1,1000.0,Staphylococcus aureus subsp. aureus,1000.0
1,C[C@H](CCC1(O)O[C@H]2C[C@H]3[C@@H]4CC[C@H]5C[C...,CHEMBL2271376,MIC,ug.mL-1,1000.0,Staphylococcus aureus subsp. aureus,1000.0
2,COC1(CC[C@@H](C)CO[C@@H]2O[C@H](CO)[C@@H](O)[C...,CHEMBL2271375,MIC,ug.mL-1,1000.0,Staphylococcus aureus subsp. aureus,1000.0


In [ ]:
df['target_organism'].value_counts()

Staphylococcus aureus subsp. aureus    937
Name: target_organism, dtype: int64

In [ ]:
df['target_organism'] = 'Staph. aureus'
df.head(3)

,canonical_smiles,molecule_chembl_id,standard_type,standard_units,standard_value,target_organism,value
0,C=C[C@](C)(CC/C=C(\C)CC/C=C(\C)CC/C=C(/C)CO[C@...,CHEMBL2271377,MIC,ug.mL-1,1000.0,Staph. aureus,1000.0
1,C[C@H](CCC1(O)O[C@H]2C[C@H]3[C@@H]4CC[C@H]5C[C...,CHEMBL2271376,MIC,ug.mL-1,1000.0,Staph. aureus,1000.0
2,COC1(CC[C@@H](C)CO[C@@H]2O[C@H](CO)[C@@H](O)[C...,CHEMBL2271375,MIC,ug.mL-1,1000.0,Staph. aureus,1000.0


## Handling Missing Data

In [ ]:
df = df[df.standard_value.notna()]
df = df[df.canonical_smiles.notna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 905 entries, 0 to 936
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   canonical_smiles    905 non-null    object
 1   molecule_chembl_id  905 non-null    object
 2   standard_type       905 non-null    object
 3   standard_units      905 non-null    object
 4   standard_value      905 non-null    object
 5   target_organism     905 non-null    object
 6   value               905 non-null    object
dtypes: object(7)
memory usage: 56.6+ KB


In [ ]:
# Note our target variable here is the Dose concentration
# These dose conc. vary by large values, Itll be great to normalize these values

 **Calculating Lipinski descriptors**

Lipinski's Rule of Five, also known as the Lipinski's descriptors or simply Lipinski descriptors, is a set of rules and guidelines in medicinal chemistry used to assess the drug-likeness of a chemical compound. These descriptors were developed by Dr. Christopher A. Lipinski, a pharmaceutical scientist, as a way to evaluate the likelihood of a compound becoming an orally active drug.

The Lipinski descriptors are based on the idea that compounds that conform to certain physicochemical properties are more likely to be absorbed into the bloodstream and exhibit oral bioavailability. The Rule of Five includes four key descriptors, and compounds are evaluated based on these criteria:

1. **Molecular Weight (MW)**: The molecular weight of the compound should be less than 500 Daltons (Da). Compounds with excessive molecular weight can have difficulty crossing cell membranes.

2. **Lipophilicity (LogP)**: The logarithm of the partition coefficient (LogP) should be less than 5. LogP measures the compound's hydrophobicity or lipophilicity. Excessive lipophilicity can lead to poor solubility and absorption.

3. **Hydrogen Bond Donors (HBD)**: The number of hydrogen bond donors should be less than 5. Hydrogen bond donors are typically groups containing hydrogen atoms that can form hydrogen bonds with other molecules.

4. **Hydrogen Bond Acceptors (HBA)**: The number of hydrogen bond acceptors should be less than 10. Hydrogen bond acceptors are typically groups containing oxygen or nitrogen atoms that can accept hydrogen bonds.

The idea behind Lipinski's Rule of Five is that compounds that violate these rules may have a higher likelihood of exhibiting poor pharmacokinetic properties, reduced oral bioavailability, and increased potential for failure in drug development. However, it's important to note that these rules are not strict cutoffs but rather guidelines that help medicinal chemists and drug developers prioritize compounds with a higher likelihood of success.

While Lipinski's Rule of Five is a valuable tool in early drug discovery, it's important to recognize that there are exceptions, and not all successful drugs adhere strictly to these rules. It's just one of many factors to consider when evaluating the drug-likeness of a compound.

In [ ]:
 ! pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 43.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [ ]:
def calc_Lipinski_desc(smile):
  cmpds = []
  for cmpd in smile:
    cmpdx = Chem.MolFromSmiles(cmpd)

    if cmpdx:
      descriptor = {
          "cmpd"  : cmpdx,
          "Molecular wt" : Descriptors.MolWt(cmpdx),
          "LogP" : Descriptors.MolLogP(cmpdx),
          "Hydrogen Bond Donors" : Descriptors.NumHDonors(cmpdx),
          "Hydrogen Bond Donors" : Descriptors.NumHAcceptors(cmpdx),
          "Aromatic Proportion" : Descriptors.FractionCSP3(cmpdx),
          "Rotatable bonds" : Descriptors.NumRotatableBonds(cmpdx)
      }

    else:
      descriptor = {
          "cmpd"  : "Invalid_cmpd",
          "Molecular wt" : "Invalid_cmpd",
          "LogP" :"Invalid_cmpd",
          "Hydrogen Bond Donors" : "Invalid_cmpd",
          "Hydrogen Bond Donors" : "Invalid_cmpd",
          "Aromatic Proportion" : "Invalid_cmpd",
          "Rotatable bonds" : "Invalid_cmpd"
      }


    cmpds.append(descriptor)

  return pd.DataFrame(cmpds)


In [ ]:
df_Lipinski = calc_Lipinski_desc(df.canonical_smiles)

In [ ]:
df_Lipinski

,cmpd,Molecular wt,LogP,Hydrogen Bond Donors,Aromatic Proportion,Rotatable bonds
0,<rdkit.Chem.rdchem.Mol object at 0x7b4054160f20>,1085.197,-3.7286,25,0.840000,24
1,<rdkit.Chem.rdchem.Mol object at 0x7b4054160f90>,1215.340,-5.4917,28,1.000000,18
2,<rdkit.Chem.rdchem.Mol object at 0x7b4054161000>,1391.508,-7.0134,33,1.000000,22
3,<rdkit.Chem.rdchem.Mol object at 0x7b4054161070>,1377.481,-7.6675,33,1.000000,21
4,<rdkit.Chem.rdchem.Mol object at 0x7b40541610e0>,1421.534,-7.6525,34,1.000000,23
...,...,...,...,...,...,...
900,<rdkit.Chem.rdchem.Mol object at 0x7b4053921d20>,358.478,4.7784,4,0.454545,9
901,<rdkit.Chem.rdchem.Mol object at 0x7b4053921d90>,299.801,4.4247,1,0.277778,3
902,<rdkit.Chem.rdchem.Mol object at 0x7b4053921e00>,362.610,5.0159,2,0.000000,2
903,<rdkit.Chem.rdchem.Mol object at 0x7b4053921e70>,449.482,6.3530,6,0.083333,6


In [ ]:
df_staph = pd.concat([df, df_Lipinski], axis = 1)
df_staph  = df_staph.dropna(subset = ['canonical_smiles','cmpd'])
df_staph

,canonical_smiles,molecule_chembl_id,standard_type,standard_units,standard_value,target_organism,value,cmpd,Molecular wt,LogP,Hydrogen Bond Donors,Aromatic Proportion,Rotatable bonds
0,C=C[C@](C)(CC/C=C(\C)CC/C=C(\C)CC/C=C(/C)CO[C@...,CHEMBL2271377,MIC,ug.mL-1,1000.0,Staph. aureus,1000.0,<rdkit.Chem.rdchem.Mol object at 0x7b4054160f20>,1085.197,-3.7286,25.0,0.840000,24.0
1,C[C@H](CCC1(O)O[C@H]2C[C@H]3[C@@H]4CC[C@H]5C[C...,CHEMBL2271376,MIC,ug.mL-1,1000.0,Staph. aureus,1000.0,<rdkit.Chem.rdchem.Mol object at 0x7b4054160f90>,1215.340,-5.4917,28.0,1.000000,18.0
2,COC1(CC[C@@H](C)CO[C@@H]2O[C@H](CO)[C@@H](O)[C...,CHEMBL2271375,MIC,ug.mL-1,1000.0,Staph. aureus,1000.0,<rdkit.Chem.rdchem.Mol object at 0x7b4054161000>,1391.508,-7.0134,33.0,1.000000,22.0
3,C[C@H](CCC1(O)O[C@H]2C[C@H]3[C@@H]4CC[C@H]5C[C...,CHEMBL2271374,MIC,ug.mL-1,1000.0,Staph. aureus,1000.0,<rdkit.Chem.rdchem.Mol object at 0x7b4054161070>,1377.481,-7.6675,33.0,1.000000,21.0
4,COC1(CC[C@@H](C)CO[C@@H]2O[C@H](CO)[C@@H](O)[C...,CHEMBL2271373,MIC,ug.mL-1,1000.0,Staph. aureus,1000.0,<rdkit.Chem.rdchem.Mol object at 0x7b40541610e0>,1421.534,-7.6525,34.0,1.000000,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,CC[C@H](NC(=O)[C@H](C)NC)C(=O)N1C[C@@H](O)C[C@...,CHEMBL3039522,MIC,nM,20000.0,Staph. aureus,20.0,<rdkit.Chem.rdchem.Mol object at 0x7b4053921d20>,358.478,4.7784,4.0,0.454545,9.0
901,CC[C@@]1(O)C(=O)OCc2c1cc1n(c2=O)Cc2cc3c([N+](=...,CHEMBL77305,MIC,nM,20000.0,Staph. aureus,20.0,<rdkit.Chem.rdchem.Mol object at 0x7b4053921d90>,299.801,4.4247,1.0,0.277778,3.0
902,O=C(Nc1ccc(N2CCN(C(=O)c3ccccc3)CC2)cc1)c1ccc2c...,CHEMBL1469003,MIC,nM,20000.0,Staph. aureus,20.0,<rdkit.Chem.rdchem.Mol object at 0x7b4053921e00>,362.610,5.0159,2.0,0.000000,2.0
903,C[C@H]1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)...,CHEMBL33,MIC,nM,2500.0,Staph. aureus,2.5,<rdkit.Chem.rdchem.Mol object at 0x7b4053921e70>,449.482,6.3530,6.0,0.083333,6.0


In [ ]:
def convert_to_nM(df):
    # Convert 'standard_value' column to float
    df['standard_value'] = df['standard_value'].astype(float)
    # Use boolean indexing to filter rows where 'standard_units' is 'ug/mL'
    mask = df['standard_units'] == 'ug.mL-1'
    df.loc[mask, 'standard_value'] = (df.loc[mask, 'standard_value'] * 10**6) / df.loc[mask, 'Molecular wt']
    df.loc[mask, 'standard_units'] = 'nM'

convert_to_nM(df_staph)
df_staph

,canonical_smiles,molecule_chembl_id,standard_type,standard_units,standard_value,target_organism,value,cmpd,Molecular wt,LogP,Hydrogen Bond Donors,Aromatic Proportion,Rotatable bonds
0,C=C[C@](C)(CC/C=C(\C)CC/C=C(\C)CC/C=C(/C)CO[C@...,CHEMBL2271377,MIC,nM,921491.673862,Staph. aureus,1000.0,<rdkit.Chem.rdchem.Mol object at 0x7b4054160f20>,1085.197,-3.7286,25.0,0.840000,24.0
1,C[C@H](CCC1(O)O[C@H]2C[C@H]3[C@@H]4CC[C@H]5C[C...,CHEMBL2271376,MIC,nM,822815.014728,Staph. aureus,1000.0,<rdkit.Chem.rdchem.Mol object at 0x7b4054160f90>,1215.340,-5.4917,28.0,1.000000,18.0
2,COC1(CC[C@@H](C)CO[C@@H]2O[C@H](CO)[C@@H](O)[C...,CHEMBL2271375,MIC,nM,718644.808366,Staph. aureus,1000.0,<rdkit.Chem.rdchem.Mol object at 0x7b4054161000>,1391.508,-7.0134,33.0,1.000000,22.0
3,C[C@H](CCC1(O)O[C@H]2C[C@H]3[C@@H]4CC[C@H]5C[C...,CHEMBL2271374,MIC,nM,725962.826348,Staph. aureus,1000.0,<rdkit.Chem.rdchem.Mol object at 0x7b4054161070>,1377.481,-7.6675,33.0,1.000000,21.0
4,COC1(CC[C@@H](C)CO[C@@H]2O[C@H](CO)[C@@H](O)[C...,CHEMBL2271373,MIC,nM,703465.411309,Staph. aureus,1000.0,<rdkit.Chem.rdchem.Mol object at 0x7b40541610e0>,1421.534,-7.6525,34.0,1.000000,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,CC[C@H](NC(=O)[C@H](C)NC)C(=O)N1C[C@@H](O)C[C@...,CHEMBL3039522,MIC,nM,20000.000000,Staph. aureus,20.0,<rdkit.Chem.rdchem.Mol object at 0x7b4053921d20>,358.478,4.7784,4.0,0.454545,9.0
901,CC[C@@]1(O)C(=O)OCc2c1cc1n(c2=O)Cc2cc3c([N+](=...,CHEMBL77305,MIC,nM,20000.000000,Staph. aureus,20.0,<rdkit.Chem.rdchem.Mol object at 0x7b4053921d90>,299.801,4.4247,1.0,0.277778,3.0
902,O=C(Nc1ccc(N2CCN(C(=O)c3ccccc3)CC2)cc1)c1ccc2c...,CHEMBL1469003,MIC,nM,20000.000000,Staph. aureus,20.0,<rdkit.Chem.rdchem.Mol object at 0x7b4053921e00>,362.610,5.0159,2.0,0.000000,2.0
903,C[C@H]1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)...,CHEMBL33,MIC,nM,2500.000000,Staph. aureus,2.5,<rdkit.Chem.rdchem.Mol object at 0x7b4053921e70>,449.482,6.3530,6.0,0.083333,6.0


**Converting IC50 to pIC50 and Normalizing the values**

**Convert IC50 to pIC50**
IC50 (half-maximal inhibitory concentration) and pIC50 (negative logarithm of IC50) are both commonly used in pharmacology and drug discovery to describe the potency or efficacy of a drug or compound. The conversion from IC50 to pIC50 is done to simplify data analysis and provide a more linear relationship to better represent the variation in potency.

Here are the reasons for the conversion from IC50 to pIC50:

1. **Linearity**: The pIC50 scale is logarithmic, which means that a one-unit change on the pIC50 scale corresponds to a tenfold change in potency. This linearization makes it easier to compare the relative potencies of different compounds. In contrast, the IC50 scale is nonlinear and can lead to difficulties in comparing compounds with a wide range of potencies.

2. **Normalization**: The pIC50 scale normalizes the data, making it suitable for statistical analysis and modeling. It ensures that the data meets the assumptions of normality and homoscedasticity often required for regression and other statistical methods.

3. **Ease of Interpretation**: The pIC50 scale provides a more intuitive way to understand and communicate the potency of a compound. For example, a pIC50 of 7 is ten times more potent than a pIC50 of 6.

4. **Consistency**: Using pIC50 allows for a consistent scale when comparing different studies or datasets. IC50 values can vary significantly between different experimental conditions, making direct comparisons challenging.

The conversion from IC50 to pIC50 is done using the following formula:

pIC50 = -log10(IC50)

Where:
- pIC50 is the negative logarithm of IC50.
- IC50 is the half-maximal inhibitory concentration.

It's important to note that while pIC50 is a valuable transformation for many applications, it might not be suitable for all cases. For extremely potent or extremely weak compounds, the transformation may not accurately represent the data. Therefore, researchers should be aware of the limitations and context-specific considerations when working with pIC50 data.

In [ ]:
import numpy as np
def pIC50(df_p):
  pIC50_d = []
  for dose in df_p['standard_value']:
    dose_float = np.array(dose, dtype=float)
    pIC50_d.append(-np.log10(dose_float * 10**-6))

  df_p['pIC50'] = pIC50_d
  return df_p


In [ ]:
df_staph1 = pIC50(df_staph)
df_staph1.reset_index(drop=True, inplace=True)
df_staph1.info()

In [ ]:
df_staph1['pIC50'].value_counts()

 **Calculating Molecular Descriptors**

Molecular descriptors are quantitative representations of the chemical and physical properties of a molecule. These descriptors are used in various fields of chemistry, drug discovery, and computational chemistry to characterize and analyze molecules, including small organic compounds, polymers, and biomolecules. Molecular descriptors provide a means to convert complex chemical structures into numerical values, allowing for the comparison and analysis of chemical compounds.

In [ ]:
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh

--2023-10-29 14:07:10--  https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip [following]
--2023-10-29 14:07:10--  https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25768637 (25M) [application/zip]
Saving to: ‘padel.zip’

padel.zip           100%[===================>]  24.57M   148MB/s    in 0.2s    

2023-10-29 14:07:10 (148 MB/s) - ‘padel.zip’ saved [25768637/25768637]

--2023-10-29 14:07:10--  https://github

In [ ]:
! unzip padel.zip

Archive:  padel.zip
   creating: PaDEL-Descriptor/
  inflating: __MACOSX/._PaDEL-Descriptor  
  inflating: PaDEL-Descriptor/MACCSFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._MACCSFingerprinter.xml  
  inflating: PaDEL-Descriptor/AtomPairs2DFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._AtomPairs2DFingerprinter.xml  
  inflating: PaDEL-Descriptor/EStateFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._EStateFingerprinter.xml  
  inflating: PaDEL-Descriptor/Fingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._Fingerprinter.xml  
  inflating: PaDEL-Descriptor/.DS_Store  
  inflating: __MACOSX/PaDEL-Descriptor/._.DS_Store  
   creating: PaDEL-Descriptor/license/
  inflating: __MACOSX/PaDEL-Descriptor/._license  
  inflating: PaDEL-Descriptor/KlekotaRothFingerprintCount.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._KlekotaRothFingerprintCount.xml  
  inflating: PaDEL-Descriptor/config  
  inflating: __MACOSX/PaDEL-Descriptor/._config  
  inf

**Calculating the  Fingerprint/PaDEL descriptors**

In [ ]:
 selection = ['canonical_smiles', 'molecule_chembl_id']
 df_staph1_select = df_staph1[selection]
 df_staph1_select.to_csv('molecule.smi', sep = '\t', index  = False, header = False)

In [ ]:
! cat molecule.smi | head -5

In [ ]:
! cat molecule.smi | wc -l

In [ ]:
! bash padel.sh

Processing CHEMBL2271377 in molecule.smi (1/873). 
Processing CHEMBL2271376 in molecule.smi (2/873). 
Processing CHEMBL2271375 in molecule.smi (3/873). Average speed: 6.50 s/mol.
Processing CHEMBL2271374 in molecule.smi (4/873). Average speed: 5.74 s/mol.
Processing CHEMBL2271373 in molecule.smi (5/873). Average speed: 4.50 s/mol.
Processing CHEMBL2271372 in molecule.smi (6/873). Average speed: 3.99 s/mol.
Processing CHEMBL2271371 in molecule.smi (7/873). Average speed: 3.53 s/mol.
Processing CHEMBL2271370 in molecule.smi (8/873). Average speed: 3.44 s/mol.
Processing CHEMBL2271379 in molecule.smi (9/873). Average speed: 3.85 s/mol.
Processing CHEMBL3359151 in molecule.smi (11/873). Average speed: 2.98 s/mol.
Processing CHEMBL2271378 in molecule.smi (10/873). Average speed: 3.28 s/mol.
Processing CHEMBL3359152 in molecule.smi (12/873). Average speed: 2.71 s/mol.
Processing CHEMBL3359153 in molecule.smi (13/873). Average speed: 2.50 s/mol.
Processing CHEMBL3359154 in molecule.smi (14/87

A PubChem fingerprint, also known as a PubChem Substructure Fingerprint, is a binary fingerprint representation of a chemical compound's structural features. It is used for compound comparison, similarity searching, and chemical informatics applications. PubChem is a free and comprehensive chemical database provided by the National Center for Biotechnology Information (NCBI), a part of the National Institutes of Health (NIH).

The PubChem fingerprint is created by identifying and encoding the presence or absence of specific structural features or substructures within a chemical compound. These substructures can include functional groups, ring systems, and other chemical motifs. Each substructure is associated with a particular bit (0 or 1) in the fingerprint.

Key characteristics of PubChem fingerprints include:

1. **Binary Representation**: Each bit in the fingerprint corresponds to the presence (1) or absence (0) of a specific substructure within the molecule. A compound's fingerprint is a binary vector of fixed length.

2. **Fixed-Length**: PubChem fingerprints are typically of fixed length, and the length may vary depending on the version of the PubChem database and the specific fingerprint type being used.

3. **Substructure-Based**: The fingerprints are generated based on the identification of substructures within the chemical compound. Substructures are predefined and stored in a dictionary.

4. **Similarity Comparison**: PubChem fingerprints are often used for compound similarity searching. Similarity measures, such as Tanimoto coefficients, can be calculated to compare the similarity of two compounds based on their fingerprints.

5. **Chemoinformatics Applications**: PubChem fingerprints are used in chemoinformatics and computational chemistry for tasks like compound clustering, virtual screening, and structure-activity relationship (SAR) analysis.

6. **Multiple Types**: PubChem provides several fingerprint types, each with a different set of substructures and characteristics. Some common types include PubChem Substructure Fingerprint (PubChemFP), PubChem Extended Fingerprint (PubChem XFP), and others.

These fingerprints are valuable for organizing and searching large chemical databases, including the PubChem database itself, which contains a vast collection of chemical compounds and associated data. Researchers can use PubChem fingerprints to find compounds with similar structural features or properties, aiding in drug discovery, chemical classification, and other chemoinformatics tasks.

PubChem offers web services and tools for accessing and working with these fingerprints, making it a valuable resource for researchers in the field of chemistry and drug discovery.

In [ ]:
# running ! bash padel.sh created a .csv file containing the PubChem fingerprint and saved it in the directory

In [ ]:
df_Mole_descr = pd.read_csv('descriptors_output.csv')
df_Mole_descr = df_Mole_descr.drop(columns = ['Name'])
df_Mole_descr.reset_index(drop=True, inplace=True)
df_Mole_descr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Columns: 881 entries, PubchemFP0 to PubchemFP880
dtypes: int64(881)
memory usage: 5.9 MB


In [ ]:
df_staph_Y  = df_staph1['pIC50']
df_staph_Y.reset_index(drop=True, inplace=True)
df_staph_Y


0      0.035509
1      0.084698
2      0.143486
3      0.139086
4      0.152757
         ...   
868    1.698970
869    1.698970
870    1.698970
871    2.602060
872    1.698970
Name: pIC50, Length: 873, dtype: float64

In [ ]:
df_staph_X = df_Mole_descr
df_staph_X.reset_index(drop=True, inplace=True)
df_staph_X

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
869,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
870,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
871,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_staph_X

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
869,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
870,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
871,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#df_staph_X.drop(columns = ['canonical_smiles','molecule_chembl_id', 'standard_type','standard_units',  'target_organism', 'value', 'standard_value', 'cmpd', 'pIC50'], inplace = True)
#df_staph_X.drop(columns = ['Molecular wt','LogP','Hydrogen Bond Donors','Rotatable bonds', 'Aromatic Proportion'], inplace = True)

In [ ]:
df_staph_X

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
869,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
870,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
871,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_staph_X.to_csv('Staphylococcus_aureus_independent.csv', index = False)
df_staph_Y.to_csv('Staphylococcus_aureus_dependent.csv')

print( f' X Shape: { df_staph_X.shape}')
print( '\n')
print( f' Y Shape: { df_staph_Y.shape}')

 X Shape: (873, 881)


 Y Shape: (873,)


**Predictive algorithm building**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
# to reduce dimension of data we would remove low variance features
from sklearn.feature_selection import VarianceThreshold

In [ ]:
#variance_model = VarianceThreshold((.8 *(1-.8)))
#X = variance_model.fit_transform(df_staph_X)
X = df_staph_X
Y = df_staph_Y
Y

0      0.035509
1      0.084698
2      0.143486
3      0.139086
4      0.152757
         ...   
868    1.698970
869    1.698970
870    1.698970
871    2.602060
872    1.698970
Name: pIC50, Length: 873, dtype: float64

In [ ]:
#Data split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.10)

In [ ]:
X_train.shape, X_test.shape

((785, 881), (88, 881))

In [ ]:
model = RandomForestRegressor(n_estimators = 100)
model.fit(X_train, Y_train)
r2 = model.score(X_test, Y_test)
r2

0.0324567699860171

In [ ]:
! pip install lazypredict
import lazypredict
from lazypredict.Supervised import LazyRegressor

In [ ]:
clf = LazyRegressor(verbose = 0, ignore_warnings = True, custom_metric = None)
train, test = clf.fit(X_train, X_test, Y_train, Y_test)

In [ ]:
  #performance on train data
  train

In [ ]:
test

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
Lars,53652574473631467139235840.00,-489656829104176836647059456.00,14120749109498.68,0.57
LinearRegression,591813210381073121280.00,-5401145851064046583808.00,46898049569.86,0.89
TransformedTargetRegressor,591813210381073121280.00,-5401145851064046583808.00,46898049569.86,0.47
SGDRegressor,14422116.97,-131622528.70,7321.11,0.17
KernelRidge,2.21,-10.03,2.12,0.12
GaussianProcessRegressor,2.01,-8.18,1.93,2.24
LinearSVR,1.28,-1.57,1.02,1.77
MLPRegressor,1.24,-1.17,0.94,3.64
Ridge,1.23,-1.06,0.92,0.22


In [ ]:
tr = pd.read_csv('ty.csv')
tr

FileNotFoundError: ignored

In [ ]:
model.predict(tr)

array([2.3381193, 1.9285346, 1.9863644, 1.6744074, 2.2251399],
      dtype=float32)

In [ ]:
import pickle
with open('staph_RFR.pkl', 'wb') as file:
    pickle.dump(model, file)


In [ ]:
import joblib

# Save the model
joblib.dump(model, 'staph_RFR.pkl')

# Load the model
#loaded_model = joblib.load('model.pkl')


['staph_RFR.pkl']

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import joblib


In [ ]:
# Create an XGBoost regressor model
model = xgb.XGBRegressor(n_estimators=100)

# Fit the model to the training data
model.fit(X_train, Y_train)

# Make predictions on the test data
Y_pred = model.predict(X_test)

# Calculate the R-squared score
r2 = r2_score(Y_test, Y_pred)
print(f"R-squared score: {r2}")

# Save the trained model to a file using joblib
joblib.dump(model, 'XGR.pkl')


R-squared score: -0.15215429672923664


['XGR.pkl']